In [1]:
import torch 
import numpy as np

In [3]:
# Device configuration
print(f"Is MPS (Metal Performance Shader) built? {torch.backends.mps.is_built()}")
print(f"Is MPS available? {torch.backends.mps.is_available()}")

device = "mps" if torch.backends.mps.is_available() else "cpu"
device = torch.device(device)
print(f"Using device: {device}")

Is MPS (Metal Performance Shader) built? True
Is MPS available? True
Using device: mps


**Tensors initialization** 

In [4]:
numpy_array=np.array([[1,2],[3,4]])
tensor=torch.from_numpy(numpy_array)
original_numpy_array=tensor.numpy()

print(numpy_array)
print(tensor)
print(original_numpy_array)

[[1 2]
 [3 4]]
tensor([[1, 2],
        [3, 4]])
[[1 2]
 [3 4]]


**Tensors' operations**

In [6]:
x1=torch.rand(2,2)
x2=torch.rand(2,2)

print(x1)
print(x2)

#not in place operation
y=x1+x2
print(y)

#in place operation
x2.add_(x1)
print(x2)

x3=torch.arange(6)
print(x3)
print("shape x3:{}".format(x3.shape))
x3=x3.view(2,3)
print(x3)
print("new shape x3:{}".format(x3.shape))

matrix_multiplication=torch.mm(x1,x2)
print(matrix_multiplication)

#first row x3
element_1_1=x3[0,0]
first_row=x3[0,:]
first_column=x3[:,0]
first_2_elements_first_row=x3[0,:2]
print(element_1_1)
print(first_row)
print(first_column)
print(first_2_elements_first_row)

tensor([[0.1059, 0.8738],
        [0.6214, 0.3253]])
tensor([[0.5506, 0.0242],
        [0.3769, 0.4935]])
tensor([[0.6565, 0.8980],
        [0.9983, 0.8187]])
tensor([[0.6565, 0.8980],
        [0.9983, 0.8187]])
tensor([0, 1, 2, 3, 4, 5])
shape x3:torch.Size([6])
tensor([[0, 1, 2],
        [3, 4, 5]])
new shape x3:torch.Size([2, 3])
tensor([[0.9418, 0.8105],
        [0.7327, 0.8244]])
tensor(0)
tensor([0, 1, 2])
tensor([0, 3])
tensor([0, 1])


**Define the Dataset**

In [8]:
from torch.utils.data import Dataset, DataLoader
from torch.nn import functional
data_array=np.random.randn(100,10)
labels_array=np.random.randint(0,2,(100,))

data=torch.from_numpy(data_array).to(torch.float32)
labels=torch.from_numpy(labels_array)
labels=functional.one_hot(labels).to(torch.float32)

class customDataset(Dataset):
    def __init__(self,data,labels):
        self.data=data
        self.labels=labels
    def __getitem__(self,idx):
        return self.data[idx],self.labels[idx]
    def __len__(self):
        return self.data.shape[0]


idx=0
custom_dataset=customDataset(data,labels)
print("total length dataset:{}".format(custom_dataset.__len__()))
print("{}th element in the dataset is:{}".format(idx,custom_dataset.__getitem__(idx)[0]))
print("with corresponding label:{}".format(custom_dataset.__getitem__(idx)[1]))



total length dataset:100
0th element in the dataset is:tensor([ 1.1508,  0.9168, -0.9319, -1.2128,  0.5201,  0.4449,  1.4035, -0.4814,
         1.3009,  0.4852])
with corresponding label:tensor([0., 1.])


**Define the Dataloader**

In [10]:
Data_loader=DataLoader(custom_dataset,batch_size=5,shuffle=False)
for x,y in Data_loader:
    print(x.shape)
    print(y.shape)

torch.Size([5, 10])
torch.Size([5, 2])
torch.Size([5, 10])
torch.Size([5, 2])
torch.Size([5, 10])
torch.Size([5, 2])
torch.Size([5, 10])
torch.Size([5, 2])
torch.Size([5, 10])
torch.Size([5, 2])
torch.Size([5, 10])
torch.Size([5, 2])
torch.Size([5, 10])
torch.Size([5, 2])
torch.Size([5, 10])
torch.Size([5, 2])
torch.Size([5, 10])
torch.Size([5, 2])
torch.Size([5, 10])
torch.Size([5, 2])
torch.Size([5, 10])
torch.Size([5, 2])
torch.Size([5, 10])
torch.Size([5, 2])
torch.Size([5, 10])
torch.Size([5, 2])
torch.Size([5, 10])
torch.Size([5, 2])
torch.Size([5, 10])
torch.Size([5, 2])
torch.Size([5, 10])
torch.Size([5, 2])
torch.Size([5, 10])
torch.Size([5, 2])
torch.Size([5, 10])
torch.Size([5, 2])
torch.Size([5, 10])
torch.Size([5, 2])
torch.Size([5, 10])
torch.Size([5, 2])


**Define a simple multy-layer perceptron**

In [11]:
from torch import nn

class MLP(nn.Module):
    def __init__(self,hidden_layers_dimension,input_dimensions,output_dimensions):
        
        super(MLP, self).__init__()
        self.fc1=nn.Linear(input_dimensions,hidden_layers_dimension)
        self.act=nn.ReLU()
        self.output_layer=nn.Linear(hidden_layers_dimension,output_dimensions)
        self.act_output=nn.Softmax()
        
    def forward(self,x):
        
        x=self.fc1(x)
        x=self.act(x)
        x=self.output_layer(x)
        x=self.act_output(x)
    
        return x

mlp=MLP(12,10,2)
result=mlp(data)
print(result.shape)

torch.Size([100, 2])


/Users/guard004/miniconda3/lib/python3.10/site-packages/torch/nn/modules/module.py:1502: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


**Define the loss function**

In [12]:
criterion=torch.nn.CrossEntropyLoss()

**Define the optimizer**

In [13]:
from torch.optim import Adam
lr=0.05
optimizer=Adam(mlp.parameters(),lr)

**Training loop**

In [14]:
import tqdm
n_epochs=20
mlp.train()
mlp.to(device)

# Training loop
for epoch in range(n_epochs):
    print("epoch:{}".format(epoch))
    loss_current_epoch=[]
    for data_inputs, data_labels in Data_loader:

        data_inputs = data_inputs.to(device)
        data_labels = data_labels.to(device)

        preds = mlp(data_inputs)
        preds = preds.squeeze(dim=1)

        loss = criterion(preds, data_labels)
        
        loss_current_epoch.append(loss.item())
        optimizer.zero_grad()
        loss.backward()

        optimizer.step()
    
    print("loss epoch:{} is:{}".format(epoch,np.mean(loss_current_epoch)))

epoch:0
loss epoch:0 is:0.7271841615438461
epoch:1
loss epoch:1 is:0.6521613299846649
epoch:2
loss epoch:2 is:0.5721628397703171
epoch:3
loss epoch:3 is:0.5452552929520607
epoch:4
loss epoch:4 is:0.5160976827144623
epoch:5
loss epoch:5 is:0.5055901303887367
epoch:6
loss epoch:6 is:0.48127494007349014
epoch:7
loss epoch:7 is:0.4657953053712845
epoch:8
loss epoch:8 is:0.4581084564328194
epoch:9
loss epoch:9 is:0.44538857936859133
epoch:10
loss epoch:10 is:0.44146683663129804
epoch:11
loss epoch:11 is:0.4398169875144958
epoch:12
loss epoch:12 is:0.4382475227117538
epoch:13
loss epoch:13 is:0.43396894335746766
epoch:14
loss epoch:14 is:0.431987951695919
epoch:15
loss epoch:15 is:0.43458140194416045
epoch:16
loss epoch:16 is:0.44978891164064405
epoch:17
loss epoch:17 is:0.48134745955467223
epoch:18
loss epoch:18 is:0.4488783165812492
epoch:19
loss epoch:19 is:0.4343885347247124


**Evaluate the model**

In [18]:
data_array_test=np.random.randn(100,10)
labels_array_test=np.random.randint(0,2,(100,))

data_test=torch.from_numpy(data_array_test).to(torch.float32)
labels_test=torch.from_numpy(labels_array_test).to(torch.float32)

mlp.eval() 
mlp.to(device)
true_preds, num_preds = 0., 0.

custom_dataset_test=customDataset(data_test,labels_test)
Data_loader_test=DataLoader(custom_dataset_test,batch_size=5,shuffle=False)



with torch.no_grad(): 
    for data_inputs, data_labels in Data_loader_test:


        data_inputs, data_labels = data_inputs.to(device), data_labels.to(device)
        preds = mlp(data_inputs)
        preds = preds.squeeze(dim=1)

        pred_labels = torch.argmax(preds,1) 

        true_preds += (pred_labels == data_labels).sum()
        num_preds += data_labels.shape[0]

acc = true_preds / num_preds
print(f"Accuracy of the model: {100.0*acc:4.2f}%")

Accuracy of the model: 53.00%
